## Import Autophagy Package

In [10]:
import importlib
import autophagy
importlib.reload(autophagy)
from autophagy import *

## Define Simulation Settings

In [11]:
simulation_settings = SimulationSettings()
simulation_settings.n_times=20_001
simulation_settings.save_interval=50
simulation_settings.domain_size=50
simulation_settings.cell_mechanics_random_travel_velocity=0.025
simulation_settings.cell_mechanics_potential_strength=0.5
simulation_settings.n_threads=6
print(simulation_settings)

SimulationSettings {
    n_cells_cargo: 1,
    n_cells_r11: 500,
    cell_dampening: 1.0,
    cell_radius_cargo: 10.0,
    cell_radius_r11: 1.0,
    cell_mechanics_interaction_range_cargo: 3.0,
    cell_mechanics_interaction_range_r11: 1.0,
    cell_mechanics_random_travel_velocity: 0.025,
    cell_mechanics_random_update_time: 50.0,
    cell_mechanics_potential_strength: 0.5,
    cell_mechanics_relative_clustering_strength: 0.03,
    dt: 0.25,
    n_times: 20001,
    save_interval: 50,
    n_threads: 6,
    domain_size: 50.0,
    storage_name: "out/autophagy",
    show_progressbar: true,
}


## Actually run the Simulation

In [12]:
output_path = run_simulation(simulation_settings)
# output_path = "out/autophagy/2023-10-31-00:19:21/"

Running Simulation


## Read results from json Files

In [13]:
import os
from pathlib import Path
import json

def combine_batches(run_directory):
    combined_batch = []
    for batch_file in os.listdir(run_directory):
        f = open(run_directory / batch_file)
        b = json.load(f)["data"]
        combined_batch.extend(b)
    return combined_batch

def get_cells_at_iterations(output_path):
    dir = Path(output_path) / "cell_storage/json/"
    runs = [(x, dir / x) for x in os.listdir(dir)]
    result = []
    for (n_run, run_directory) in runs:
        result.extend([{"iteration":int(n_run)} | c for c in combine_batches(run_directory)])
    return result

cells_at_iter = get_cells_at_iterations(output_path)

In [14]:
import pandas as pd

df = pd.json_normalize(cells_at_iter)
for col in df.columns:
    print(col)

iteration
identifier
element.id
element.parent_id
element.cell.mechanics.pos
element.cell.mechanics.vel
element.cell.mechanics.dampening_constant
element.cell.mechanics.mass
element.cell.mechanics.random_travel_velocity
element.cell.mechanics.random_direction_travel
element.cell.mechanics.random_update_time
element.cell.interaction.species
element.cell.interaction.cell_radius
element.cell.interaction.potential_strength
element.cell.interaction.interaction_range
element.cell.interaction.clustering_strength


## Plot Result

In [15]:
import pyvista as pv
import numpy as np
import multiprocessing as mp

def generate_snapshot(df, iteration, odir=Path(".")):
    df_filtered = df[df["iteration"]==iteration]

    pset = pv.PolyData(np.array([np.array(x) for x in df_filtered["element.cell.mechanics.pos"]]))

    pset.point_data["diameter"] = 2.0*df_filtered["element.cell.interaction.cell_radius"]
    pset.point_data["species"] = df_filtered["element.cell.interaction.species"]

    sphere = pv.Sphere()
    spheres = pset.glyph(geom=sphere, scale="diameter", orient=False)

    plotter = pv.Plotter()
    
    spheres.plot(
        off_screen=True,
        screenshot=odir / "snapshot_{:08}.png".format(iteration),
        scalars="species",
        scalar_bar_args={
            "title":"Species",
        },
        cpos=[
            (
                -1.5*simulation_settings.domain_size,
                -1.5*simulation_settings.domain_size,
                -1.5*simulation_settings.domain_size
            ),(
                50,
                50,
                50
            ),(
                0.0,
                0.0,
                0.0
            )
        ],
        jupyter_backend='none',
    )

def save_snapshot(iteration):
    generate_snapshot(df, iteration, odir=Path(output_path))

with mp.Pool() as p:
    p.map(save_snapshot, np.unique(df["iteration"]))
